## Imports

In [6]:
import numpy as np
import librosa
import librosa.display as lbd
import pydub
import os

## Set Directories

In [8]:
#inputs
main_bayan = "Edited_25-108-Bayan-ul-Quran.mp3"
recitation_dir = "./recitations/"
timestamps_file = "./Time Stamps 25 - 108.txt"

#outputs
bayans_dir = './Bayans/'
bayan_plus_recitation_dir = './RB/'

### Function for Auto

In [9]:
def hr_min_sec_to_samples(hr, min, sec, msec, sr=44100):
    return int((hr*3600+min*60+sec+msec/1000)*sr)

def generate_audios(ayah_number, start_time, end_time, y_bayan, sr_bayan):
    
    print("\n\n================================================")
    print("Ayah Number: ", ayah_number)
    
    start_hr, start_min, start_sec, start_milli = start_time
    end_hr, end_min, end_sec, end_milli = end_time
    
    y_recitation, sr_recitation = librosa.load(recitation_dir+ayah_number+".mp3", sr = 44100, mono=False)

    ## Extract bayaan
    y_bayan_ext = y_bayan[:, hr_min_sec_to_samples(start_hr,start_min,start_sec,start_milli,sr_bayan):hr_min_sec_to_samples(end_hr,end_min,end_sec,end_milli,sr_bayan)].copy(order='F')

    print("Duration of Bayan: ", librosa.get_duration(y_bayan_ext, sr = sr_bayan)/60, "minutes")
    print("Duration of Recitation: ", librosa.get_duration(y_recitation, sr = sr_recitation), "sec")

    ## Save Bayan 
    librosa.output.write_wav('./'+ayah_number+'_Bayan.wav', y_bayan_ext, sr_bayan)
    pydub.AudioSegment.from_wav('./'+ayah_number+'_Bayan.wav').export(bayans_dir+ayah_number+'_Bayan.mp3', format='mp3', bitrate="192k")
    os.remove('./'+ayah_number+'_Bayan.wav')
    
    ## combine them
    y_combined = np.concatenate([y_recitation,y_bayan_ext],axis=1)
    sr_combined = 44100
    
    ## Max Normalize
    y_combined /= np.max(np.abs(y_combined), axis=1).reshape(-1,1) #Normalize (0-1)
    
    ## Hard limit at 0.50
    np.clip(y_combined,-0.50,0.50,y_combined)

    ## Max Normalize
    y_combined /= np.max(np.abs(y_combined), axis=1).reshape(-1,1) #Normalize (0-1)
    
    ## Save Bayan + Recitation
    librosa.output.write_wav('./'+ayah_number+'_RB.wav', y_combined, sr_combined)
    pydub.AudioSegment.from_wav('./'+ayah_number+'_RB.wav').export(bayan_plus_recitation_dir+ayah_number+'_RB.mp3', format='mp3', bitrate="192k")
    os.remove('./'+ayah_number+'_RB.wav')
    
def generate_tamheed(tamheed_name, start_time, end_time, y_bayan, sr_bayan):
    
    print("\n\n================================================")
    print("Tamheed Name: ", tamheed_name)
    
    start_hr, start_min, start_sec, start_milli = start_time
    end_hr, end_min, end_sec, end_milli = end_time

    ## Extract tamheed
    y_bayan_ext = y_bayan[:, hr_min_sec_to_samples(start_hr,start_min,start_sec,start_milli,sr_bayan):hr_min_sec_to_samples(end_hr,end_min,end_sec,end_milli,sr_bayan)].copy(order='F')

    print("Duration of Tamheed: ", librosa.get_duration(y_bayan_ext, sr = sr_bayan)/60, "minutes")

    ## Max Normalize
    y_bayan_ext /= np.max(np.abs(y_bayan_ext), axis=1).reshape(-1,1) #Normalize (0-1)

    ## Hard limit at 0.50
    np.clip(y_bayan_ext,-0.50,0.50,y_bayan_ext)
    
    ## Max Normalize again
    y_bayan_ext /= np.max(np.abs(y_bayan_ext), axis=1).reshape(-1,1) #Normalize (0-1)
    
    ## Save Tamheed
    librosa.output.write_wav('./'+tamheed_name+'_Bayan.wav', y_bayan_ext, sr_bayan)
    pydub.AudioSegment.from_wav('./'+tamheed_name+'_Bayan.wav').export(bayans_dir+tamheed_name+'_Bayan.mp3', format='mp3', bitrate="192k")
    os.remove('./'+tamheed_name+'_Bayan.wav')

### Automatic Timestamps Reading

In [10]:
with open(timestamps_file) as f:
    timeStamps = [line.strip().split() for line in f.readlines() if line.strip()]
    
ayah_numbers = [t[0] for t in timeStamps]
start_times = [[int(string) for string in line[1].split(':')] for line in timeStamps]
end_times = [[int(string) for string in line[2].split(':')] for line in timeStamps]

## Looping over all

In [11]:
#Load the main bayan
y_bayan, sr_bayan = librosa.load(main_bayan, sr = 44100, mono=False)

In [12]:
%%time
#Loop over all the ayahs
for ayah_number, start_time, end_time in zip(ayah_numbers, start_times, end_times):
    try:
        int(ayah_number)
        generate_audios(ayah_number, start_time, end_time, y_bayan, sr_bayan)
    except ValueError:
        generate_tamheed(ayah_number, start_time, end_time, y_bayan, sr_bayan)



Ayah Number:  004142
Duration of Bayan:  5.282333333333333 minutes
Duration of Recitation:  25.30825396825397 sec


Ayah Number:  004143
Duration of Bayan:  1.074 minutes
Duration of Recitation:  23.729160997732425 sec


Ayah Number:  004144
Duration of Bayan:  0.7866999244142101 minutes
Duration of Recitation:  24.71077097505669 sec


Ayah Number:  004145
Duration of Bayan:  0.6515000000000001 minutes
Duration of Recitation:  11.224399092970522 sec


Ayah Number:  004146
Duration of Bayan:  2.38519992441421 minutes
Duration of Recitation:  25.09487528344671 sec


Ayah Number:  004147
Duration of Bayan:  1.0926500377928952 minutes
Duration of Recitation:  14.425283446712019 sec


Ayah Number:  004148
Duration of Bayan:  1.7500332577475435 minutes
Duration of Recitation:  12.718140589569161 sec


Ayah Number:  004149
Duration of Bayan:  1.0961334089191233 minutes
Duration of Recitation:  14.51063492063492 sec


Ayah Number:  004150
Duration of Bayan:  2.3133333333333335 minutes
Durati

In [13]:
print(librosa.__version__)

0.6.0
